https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/planetoid.html
 
https://github.com/facebookresearch/SEAL_OGB/issues/1
       https://nbviewer.jupyter.org/github/NIRVANALAN/gcn_analysis/blob/master/notebook/Plantenoid%20Citation%20Data%20Format%20Transformation.ipynb
       
       
cd /Users/chaupham/minhchau/jupyter_projects/GRAPH/notebooks/SEAL_OGB-master/


python seal_link_pred.py --dataset Cora --num_hops 2 --use_feature --runs 5
python seal_link_pred.py --dataset CitationGraph --num_hops 2 --use_feature --runs 5

python seal_link_pred.py --dataset GeneGraph_withNodeFeatures --num_hops 2 --use_feature --runs 2

python seal_link_pred.py --dataset GeneGraph_withNodeFeatures_Zscore --num_hops 2 --use_feature --runs 2

python seal_link_pred.py --dataset mydataset_gene_withNodeFeatures_Rnorm_CPM --num_hops 2 --use_feature --runs 2



python seal_link_pred.py --dataset GeneGraph --num_hops 2 --runs 1




In [4]:
pip install torch-sparse

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Copyright (c) Facebook, Inc. and its affiliates.
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import torch
import pandas as pd
import argparse
import math
import time
import random
import os, sys
import os.path as osp
from itertools import chain
from shutil import copy
import copy as cp
from tqdm import tqdm
import pdb
from sklearn import preprocessing, feature_extraction, model_selection
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import scipy.sparse as ssp
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.nn import ModuleList, Linear, Conv1d, MaxPool1d, Embedding
from torch.utils.data import DataLoader

import torch
# from torch_sparse import coalesce
# from torch_scatter import scatter_min
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, SAGEConv, global_sort_pool, global_add_pool
from torch_geometric.data import Data, Dataset, InMemoryDataset, DataLoader
from torch_geometric.utils import (negative_sampling, add_self_loops,
                                   train_test_split_edges, to_networkx, 
                                   to_scipy_sparse_matrix, to_undirected)

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

import warnings
from scipy.sparse import SparseEfficiencyWarning
warnings.simplefilter('ignore',SparseEfficiencyWarning)

import os.path as osp

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.data import Data

from torch_geometric.nn import GCNConv, GNNExplainer
import dill

from utils import *


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 199)

In [17]:
def do_edge_split(dataset):
    data = dataset[0]
    data = train_test_split_edges(data)

    edge_index, _ = add_self_loops(data.train_pos_edge_index)
    data.train_neg_edge_index = negative_sampling(
        edge_index, num_nodes=data.num_nodes,
        num_neg_samples=data.train_pos_edge_index.size(1))

    split_edge = {'train': {}, 'valid': {}, 'test': {}}
    split_edge['train']['edge'] = data.train_pos_edge_index.t()
    split_edge['train']['edge_neg'] = data.train_neg_edge_index.t()
    split_edge['valid']['edge'] = data.val_pos_edge_index.t()
    split_edge['valid']['edge_neg'] = data.val_neg_edge_index.t()
    split_edge['test']['edge'] = data.test_pos_edge_index.t()
    split_edge['test']['edge_neg'] = data.test_neg_edge_index.t()
    return split_edge


In [ ]:
path = "../my_data/"

In [18]:
df=pd.read_csv(path + "gene_edges.csv")

print(df.shape)
df.head()


(657433, 2)


,gene1,gene2
0,gngt2,gngt1
1,cfl1,serpinh1
2,cfl2,cfl1
3,luzp4,cfl2
4,ldha,ldhb


In [19]:
len(set.union(set(df.gene1), set(df.gene2)))

14310

In [20]:
df_features = pd.read_csv(path + "pseudocounts_AVG_14k_GENES.csv")

all_genes = df_features.gene.values
print(len(all_genes))

14320


In [21]:
df_features.head()

,gene,P53KO.O1,p53KO.O2,p53KO.O.CAS1,p53KO.O.CAS2,p53KO.O.RAS1,p53KO.O.RAS2,WT.O1,WT.O2
0,hoxc10,2.775558e-17,2.775558e-17,1.588892e+00,0.000000e+00,2.775558e-17,2.775558e-17,2.775558e-17,2.775558e-17
1,calcr,2.456023e+00,1.223430e-02,2.775558e-17,2.775558e-17,4.508758e+00,3.743896e+00,1.760702e-01,4.042754e+00
2,rabggtb,3.097384e+03,3.286204e+03,2.488481e+03,2.141901e+03,2.808107e+03,3.228427e+03,2.786014e+03,3.280963e+03
3,sptlc3,2.775558e-17,2.775558e-17,2.775558e-17,2.775558e-17,1.095702e+00,9.318960e-01,2.775558e-17,2.775558e-17
4,ap3d1,3.682757e+03,4.298682e+03,4.098192e+03,3.370365e+03,4.096048e+03,3.754787e+03,4.159898e+03,4.005069e+03


In [22]:
dict_num_to_name = dict(zip( list(range(len(all_genes))), all_genes))
dict_name_to_num = dict(zip( all_genes, list(range(len(all_genes))) ))

In [23]:
edges = np.array([df.gene1.map(dict_name_to_num).values, df.gene2.map(dict_name_to_num).values.astype(int)])
edge_index = torch.from_numpy(edges)
edge_index

tensor([[ 8825, 11337, 11789,  ..., 13502, 13502, 13502],
        [ 6226,  9844, 11337,  ..., 12524, 13069,   376]])

In [24]:
len(edge_index[0])

657433

In [25]:
df_features.columns

Index(['gene', 'P53KO.O1', 'p53KO.O2', 'p53KO.O.CAS1', 'p53KO.O.CAS2', 'p53KO.O.RAS1', 'p53KO.O.RAS2', 'WT.O1', 'WT.O2'], dtype='object')

In [26]:
filtered_cols = ['P53KO.O1', 'p53KO.O2', 'p53KO.O.CAS1', 'p53KO.O.CAS2', 'p53KO.O.RAS1', 'p53KO.O.RAS2', 'WT.O1', 'WT.O2']
# avg_cols = list()
# for i in range(0, len(filtered_cols),2):
#     new_col = filtered_cols[i].replace("1", "_avg")
#     avg_cols.append(new_col)
#     df_features[new_col] = (df_features[filtered_cols[i]] + df_features[filtered_cols[i+1]])/2

avg_cols = filtered_cols
df_features = df_features[avg_cols]

In [27]:
for c in df_features.columns:
    sum_col = df_features[c].sum()
    df_features[c] = df_features[c]*10**6/sum_col
df_features

,P53KO.O1,p53KO.O2,p53KO.O.CAS1,p53KO.O.CAS2,p53KO.O.RAS1,p53KO.O.RAS2,WT.O1,WT.O2
0,5.932999e-19,7.225862e-19,4.879189e-02,0.000000e+00,5.660188e-19,4.726793e-19,4.602508e-19,4.587818e-19
1,5.249966e-02,3.185067e-04,8.523218e-19,7.816656e-19,9.194699e-02,6.375880e-02,2.919646e-03,6.682412e-02
2,6.620932e+01,8.555274e+01,7.641660e+01,6.032122e+01,5.726566e+01,5.498033e+01,4.619847e+01,5.423221e+01
3,5.932999e-19,7.225862e-19,8.523218e-19,7.816656e-19,2.234462e-02,1.587025e-02,4.602508e-19,4.587818e-19
4,7.872216e+01,1.119115e+02,1.258478e+02,9.491780e+01,8.353061e+01,6.394428e+01,6.898061e+01,6.620120e+01
...,...,...,...,...,...,...,...,...
14315,2.980688e+01,4.036827e+01,3.013039e+01,2.452363e+01,3.263333e+01,3.248617e+01,3.028531e+01,3.822892e+01
14316,2.859665e-01,4.887360e-01,2.173790e+00,1.934800e+00,3.507695e-01,1.909479e-01,1.897462e-01,1.489797e-01
14317,1.085178e+02,1.389250e+02,7.160371e+01,6.056817e+01,2.018757e+02,1.928608e+02,1.998530e+02,1.950146e+02
14318,5.846910e-02,1.085512e-01,4.355452e-02,7.430233e-02,4.557069e-02,3.182916e-02,1.172446e-01,7.661417e-02


In [28]:
for c in df_features.columns:
    max_col = df_features[c].max()
    df_features[c] = df_features[c]/max_col
df_features

,P53KO.O1,p53KO.O2,p53KO.O.CAS1,p53KO.O.CAS2,p53KO.O.RAS1,p53KO.O.RAS2,WT.O1,WT.O2
0,9.664070e-24,1.343973e-23,9.071805e-07,0.000000e+00,5.259538e-24,4.107615e-24,3.983451e-24,3.380895e-24
1,8.551500e-07,5.924062e-09,1.584710e-23,1.407500e-23,8.543862e-07,5.540682e-07,2.526941e-08,4.924462e-07
2,1.078462e-03,1.591237e-03,1.420803e-03,1.086169e-03,5.321217e-04,4.777828e-04,3.998458e-04,3.996527e-04
3,9.664070e-24,1.343973e-23,1.584710e-23,1.407500e-23,2.076298e-07,1.379136e-07,3.983451e-24,3.380895e-24
4,1.282280e-03,2.081497e-03,2.339870e-03,1.709130e-03,7.761799e-04,5.556801e-04,5.970242e-04,4.878557e-04
...,...,...,...,...,...,...,...,...
14315,4.855147e-04,7.508292e-04,5.602100e-04,4.415829e-04,3.032342e-04,2.823070e-04,2.621181e-04,2.817199e-04
14316,4.658016e-06,9.090240e-06,4.041696e-05,3.483882e-05,3.259407e-06,1.659350e-06,1.642245e-06,1.097874e-06
14317,1.767612e-03,2.583933e-03,1.331317e-03,1.090616e-03,1.875862e-03,1.675973e-03,1.729719e-03,1.437119e-03
14318,9.523843e-07,2.018998e-06,8.098029e-07,1.337919e-06,4.234502e-07,2.765976e-07,1.014747e-06,5.645918e-07


In [29]:
feature_cols = avg_cols
len(feature_cols)

x = df_features[feature_cols].values
x = torch.from_numpy(x)
x

tensor([[9.6641e-24, 1.3440e-23, 9.0718e-07,  ..., 4.1076e-24, 3.9835e-24,
         3.3809e-24],
        [8.5515e-07, 5.9241e-09, 1.5847e-23,  ..., 5.5407e-07, 2.5269e-08,
         4.9245e-07],
        [1.0785e-03, 1.5912e-03, 1.4208e-03,  ..., 4.7778e-04, 3.9985e-04,
         3.9965e-04],
        ...,
        [1.7676e-03, 2.5839e-03, 1.3313e-03,  ..., 1.6760e-03, 1.7297e-03,
         1.4371e-03],
        [9.5238e-07, 2.0190e-06, 8.0980e-07,  ..., 2.7660e-07, 1.0147e-06,
         5.6459e-07],
        [3.7421e-04, 5.1861e-04, 6.0156e-04,  ..., 3.0526e-04, 3.4954e-04,
         2.5870e-04]], dtype=torch.float64)

In [32]:
len(edge_index[0]) -59169- 65743

657433

In [35]:


n_nodes = len(edge_index[0])
train_size =  len(edge_index[0]) -59169- 65743
val_size =  59169 #int(n_nodes * 0.2)


train_index, test_index = model_selection.train_test_split(
     list(range(len(edge_index[0]))), train_size=train_size, test_size=None,random_state = 42,
)
val_index, test_index = model_selection.train_test_split(
    test_index, train_size=val_size, test_size=None, random_state = 42,
)

train_mask = torch.tensor(train_index, dtype=torch.long)
val_mask = torch.tensor(val_index, dtype=torch.long)
test_mask = torch.tensor(test_index, dtype=torch.long)

train_val_mask = torch.cat((train_mask, val_mask), 0)

In [36]:
# data_path = '../../data/keyword_and_abstract_SentenceTransform/'


# with open(data_path+'new_edges.pkl', 'rb') as f:     
#     total_edges = dill.load(f)

# with open(data_path+'new_features.pkl', 'rb') as f:     
#     features = dill.load(f)

# with open(data_path+'new_labels.pkl', 'rb') as f:     
#     subjects = dill.load(f)
    

# ## remove the last paper and its edges, feature
# total_edges = total_edges[:-3]
# features = features[:-1]
# subjects = subjects[:-1]

# features_tensor, subjects_tensor = torch.tensor(features), torch.tensor(subjects)

In [37]:
# x = features
# subjects = pd.DataFrame({ "label": subjects})["label"]


# total_edges_1 = [e[0] for e in total_edges]
# total_edges_2 = [e[1] for e in total_edges]

# total_edges_1_2  = total_edges_1 + total_edges_2
# total_edges_2_1  = total_edges_2 + total_edges_1

# edge_index = np.array([total_edges_1_2, total_edges_2_1])
# print(edge_index.shape)


# x = torch.from_numpy(x)
# edge_index = np.array([total_edges_1_2, total_edges_2_1])

# edge_index = torch.from_numpy(edge_index)
# edge_index

In [38]:
data = Data(transform=T.NormalizeFeatures(), 
            x=x, edge_index=edge_index, 
            train_mask=train_val_mask, val_mask=val_mask, test_mask=test_mask, 
            y=None)

data

Data(edge_index=[2, 657433], test_mask=[65743], train_mask=[591690], transform=NormalizeFeatures(), val_mask=[59169], x=[14320, 8])

In [39]:
import dill

#### Write
with open('./mydataset_gene_withNodeFeatures_Rnorm_CPM.pkl', 'wb') as f:     
    dill.dump([data], f)
    
with open('./my_gene_data_withNodeFeatures_Rnorm_CPM.pkl', 'wb') as f:     
    dill.dump(data, f)

In [40]:
import os.path as osp

import torch
from torch_geometric.data import Dataset


class MyOwnDataset(Dataset):
    def __init__(self, root, name, transform=None, pre_transform=None):
        self.name = name

        super(MyOwnDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return 

    @property
    def processed_file_names(self):
        return 'data.pt'
    
    @property
    def processed_dir(self):
        return osp.join(self.root, self.name, 'processed')


    def process(self):
        
        # Read data from `raw_path`.
        #### Read
        with open(os.path.join(self.root, 'my_gene_data_withNodeFeatures_Rnorm_CPM.pkl'), 'rb') as f:     
            data = dill.load(f)            

        if self.pre_transform is not None:
            data = self.pre_transform(data)

        torch.save(data, osp.join(self.processed_dir, 'data.pt'))

    def len(self):
        return len(self.processed_file_names)

    def get(self):
        data = torch.load(osp.join(self.processed_dir, 'data.pt'))
        return data

In [41]:
my_citation_data = MyOwnDataset("./", "GeneGraph_withNodeFeatures_Rnorm_CPM")

In [44]:
!open .

In [42]:
path = osp.join('dataset', "Cora")
dataset = Planetoid(path, "Cora")

split_edge = do_edge_split(dataset)
data = dataset[0]
data.edge_index = split_edge['train']['edge'].t()


In [43]:
dataset.__dict__

{'name': 'Cora',
 'root': 'dataset/Cora',
 'transform': None,
 'pre_transform': None,
 'pre_filter': None,
 '__indices__': None,
 'data': Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708]),
 'slices': {'x': tensor([   0, 2708]),
  'edge_index': tensor([    0, 10556]),
  'y': tensor([   0, 2708]),
  'train_mask': tensor([   0, 2708]),
  'val_mask': tensor([   0, 2708]),
  'test_mask': tensor([   0, 2708])},
 '__data_list__': [Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])],
 'split': 'public'}

In [ ]:
type(dataset)

In [ ]:
data

In [ ]:
type(data)

In [ ]:
data.edge_index

In [ ]:
data.__dict__

In [ ]:
!open .

In [ ]:
pwd